In [11]:
import os 
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from pydantic import BaseModel,Field
from typing import Optional,Literal,Annotated
from typing import TypedDict
import operator
load_dotenv()



GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


In [2]:
Model = ChatGoogleGenerativeAI(model='gemini-2.5-flash',api_key=GOOGLE_API_KEY,temperature=0)

In [5]:
Model.invoke("which khan is most famous?")

AIMessage(content="Without a doubt, the most famous Khan is **Genghis Khan**.\n\nHere's why:\n\n1.  **Founder of the Mongol Empire:** He united the nomadic tribes of Mongolia and founded the largest contiguous land empire in history.\n2.  **Global Recognition:** His name is synonymous with conquest, empire-building, and a pivotal period in world history. Most people, even those not deeply familiar with history, have heard of Genghis Khan.\n3.  **Historical Impact:** His campaigns and the empire he established profoundly reshaped the political, cultural, and demographic landscape of Eurasia.\n\nWhile other Khans like **Kublai Khan** (his grandson, who conquered China and established the Yuan Dynasty, famously hosting Marco Polo) are also very well-known, Genghis Khan's name resonates globally as the original and most impactful figure.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, i

In [6]:
class EvaluationSchema(BaseModel):
    feedback :str = Field(description="Detail feedback for the essay")
    score : int  = Field(description="score out of 10",gt=0,lt=10)

structured_model = Model.with_structured_output(EvaluationSchema)

In [ ]:
essay = """
Quantum physics, also known as quantum mechanics, is the branch of physics that studies the behavior of matter and energy at the smallest scales, such as atoms and subatomic particles. Unlike classical physics, which follows predictable laws of motion, quantum physics reveals a world governed by probabilities and uncertainties. One of its central principles is the wave-particle duality, which shows that particles like electrons and photons can behave both as waves and particles depending on how they are observed. Another key concept, Heisenberg’s uncertainty principle, states that certain properties, such as position and momentum, cannot both be measured with complete precision at the same time. These ideas challenge our traditional understanding of reality and highlight the strange, counterintuitive nature of the quantum realm.
Despite its abstract and sometimes puzzling concepts, quantum physics has had profound practical impacts on technology and modern life. The development of semiconductors, which are the foundation of computers and smartphones, relies on quantum mechanics. Other technologies, such as lasers, magnetic resonance imaging (MRI), and even the emerging field of quantum computing, stem directly from quantum principles. Beyond technology, quantum physics has also influenced philosophy by raising questions about the nature of reality, observation, and the role of consciousness in measurement. In this way, quantum mechanics is not only a cornerstone of modern science but also a field that continues to expand human understanding of the universe.
"""

In [ ]:
prompt = f"evalute the language quality of the following essay and provide the score between 0 to 10. -essay {essay}"
structured_model.invoke(prompt)

EvaluationSchema(feedback='The essay demonstrates excellent language quality. It is clear, concise, and well-structured, making complex scientific concepts accessible. The vocabulary is precise and appropriate, and the sentence structures are varied, contributing to a smooth and engaging reading experience. There are no grammatical errors or awkward phrasing.', score=9)

In [44]:
class CSS_state(TypedDict):
    essay:str
    language_feedback:str
    analysis_feedback :str
    clarity_feedback : str
    overall_feedback : str
    individual_scores : Annotated[list[int],operator.add]
    avg_score : float

In [45]:
def evaluate_language(state:CSS_state):
    prompt = f"evalute the language quality of the following essay and provide the score between 0 to 10. -essay {state['essay']}"
    result=structured_model.invoke(prompt)
    return {"language_feedback":result.feedback,"individual_scores":[result.score]}

def evaluate_analysis(state:CSS_state):
    prompt = f"evalute the depth of  analysis the followig,and provide the score between 0 to 10. -essay {state['essay']}"
    result=structured_model.invoke(prompt)
    return {"analysis_feedback":result.feedback,"individual_scores":[result.score]}

def evaluate_thought(state:CSS_state):
    prompt = f"evalute the clarity of thought,and provide the score between 0 to 10. -essay {state['essay']}"
    result=structured_model.invoke(prompt)
    return {"clarity_feedback":result.feedback,"individual_scores":[result.score]}

def final_evaluate(state:CSS_state):
    # summary feedback and average
    prompt = f"based on the following feedback create the summary of feedback - feedback are {state['language_feedback']}  language feedback , {state['analysis_feedback']}- depth of analysis analysis feedback ,{state['clarity_feedback']} - clarity fedback"
    overall_summary=Model.invoke(prompt).content
    avg_score  = sum(state['individual_scores'])/len(state['individual_scores'])
    return {"overall_feedback":overall_summary,"avg_score":avg_score}


In [46]:
from langgraph.graph import StateGraph,START,END

graph = StateGraph(CSS_state)

#add noes in it
graph.add_node("evaluate_language",evaluate_language)
graph.add_node("evaluate_analysis",evaluate_analysis)
graph.add_node("evaluate_thought",evaluate_thought)
graph.add_node("final_evaluate",final_evaluate)


#lets make the edges
graph.add_edge(START,"evaluate_language")
graph.add_edge(START,"evaluate_analysis")
graph.add_edge(START,"evaluate_thought")

graph.add_edge("evaluate_language","final_evaluate")
graph.add_edge("evaluate_analysis","final_evaluate")
graph.add_edge("evaluate_thought","final_evaluate")

graph.add_edge("final_evaluate",END)


workflow = graph.compile()

In [47]:
essay = """
Quantum physics, also known as quantum mechanics, is the branch of physics that studies the behavior of matter and energy at the smallest scales, such as atoms and subatomic particles. Unlike classical physics, which follows predictable laws of motion, quantum physics reveals a world governed by probabilities and uncertainties. One of its central principles is the wave-particle duality, which shows that particles like electrons and photons can behave both as waves and particles depending on how they are observed. Another key concept, Heisenberg’s uncertainty principle, states that certain properties, such as position and momentum, cannot both be measured with complete precision at the same time. These ideas challenge our traditional understanding of reality and highlight the strange, counterintuitive nature of the quantum realm.

Despite its abstract and sometimes puzzling concepts, quantum physics has had profound practical impacts on technology and modern life. The development of semiconductors, which are the foundation of computers and smartphones, relies on quantum mechanics. Other technologies, such as lasers, magnetic resonance imaging (MRI), and even the emerging field of quantum computing, stem directly from quantum principles. Beyond technology, quantum physics has also influenced philosophy by raising questions about the nature of reality, observation, and the role of consciousness in measurement. In this way, quantum mechanics is not only a cornerstone of modern science but also a field that continues to expand human understanding of the universe.

"""

In [48]:
initial_state = {
    'essay' : essay
}

In [50]:
workflow.invoke(initial_state)

{'essay': '\nQuantum physics, also known as quantum mechanics, is the branch of physics that studies the behavior of matter and energy at the smallest scales, such as atoms and subatomic particles. Unlike classical physics, which follows predictable laws of motion, quantum physics reveals a world governed by probabilities and uncertainties. One of its central principles is the wave-particle duality, which shows that particles like electrons and photons can behave both as waves and particles depending on how they are observed. Another key concept, Heisenberg’s uncertainty principle, states that certain properties, such as position and momentum, cannot both be measured with complete precision at the same time. These ideas challenge our traditional understanding of reality and highlight the strange, counterintuitive nature of the quantum realm.\n\nDespite its abstract and sometimes puzzling concepts, quantum physics has had profound practical impacts on technology and modern life. The dev

In [51]:
essay2 = """
ok here you go — two short paragraphs written purposely bad / messy like you asked:

i was like woke up real late n couldnt find my phone n everything was a mess. teacher was yellin but i didn’t care, i just rolled outta bed n threw on some clothes, ya know? food was cold as hell but i ate it anyway cus whatever. day went by slow n boring, ppl keep talkin that same old stuff, i zoned out and doodled on my book instead.

then evening came n i tried doin homework but brain was done, couldnt focus, just scribbled some random crap. friends hit me up but i was feelin lazy so i ghosted em, sry not sry. ended up watchin dumb vids n laughing at stupid memes till sleepy took over. that’s it, nothing fancy, just one of those meh days.

"""

In [53]:
initial_state = {
    'essay' : essay2
}
workflow.invoke(initial_state)

{'essay': '\nok here you go — two short paragraphs written purposely bad / messy like you asked:\n\ni was like woke up real late n couldnt find my phone n everything was a mess. teacher was yellin but i didn’t care, i just rolled outta bed n threw on some clothes, ya know? food was cold as hell but i ate it anyway cus whatever. day went by slow n boring, ppl keep talkin that same old stuff, i zoned out and doodled on my book instead.\n\nthen evening came n i tried doin homework but brain was done, couldnt focus, just scribbled some random crap. friends hit me up but i was feelin lazy so i ghosted em, sry not sry. ended up watchin dumb vids n laughing at stupid memes till sleepy took over. that’s it, nothing fancy, just one of those meh days.\n\n',
 'language_feedback': "The essay uses extremely informal language, slang, and numerous grammatical errors, consistent with the user's request for a 'purposely bad / messy' writing style. It lacks formal structure and sophisticated vocabulary.